In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)


In [ ]:
df = pd.read_csv('../input/credit-card-customers/BankChurners.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['CLIENTNUM'], axis = 1, inplace = True)
df.head()

In [ ]:
df.describe()

In [ ]:
# To get all the categorical features

categorical_features = [features for features in df.columns if df[features].dtype == 'O']
print(len(categorical_features))
df[categorical_features].head()


In [ ]:
print(df['Attrition_Flag'].unique())
print(df['Gender'].unique())
print(df['Education_Level'].unique())
print(df['Marital_Status'].unique())
print(df['Income_Category'].unique())
print(df['Card_Category'].unique())

In [ ]:
# To convert yes and noes of a col to 1s and 0s

df['Attrition_Flag'] = df['Attrition_Flag'].apply(lambda x:1 if x == 'Attrited Customer' else 0)
df['Gender'] = df['Gender'].apply(lambda x:1 if x == 'M' else 0)
df.head()


In [ ]:
# List of numerical variables

num_features = [features for features in df.columns if df[features].dtype != 'O']
print(len(num_features))
df[num_features].head()


In [ ]:
# To find continuous numerical variables

continuous_feature = [features for features in num_features if len(df[features].unique())>25 ] 
print(len(continuous_feature))
df[continuous_feature].head()


In [ ]:
# To find discrete numerical variables

discrete_feature = [features for features in num_features if len(df[features].unique())<25 ] #less than 25 unique values to be considered discrete
print(len(discrete_feature))
df[discrete_feature].head()



In [ ]:
x = df.copy()

x.drop(['Attrition_Flag'], axis = 1, inplace = True)

y = df['Attrition_Flag']


In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
#OHE
x = pd.get_dummies(x, drop_first = True)

In [ ]:
x.head()

In [ ]:
x.shape

In [ ]:
x.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis = 1, inplace = True)
x.head()

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
prediction=rf.predict(x_test)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


In [ ]:
#Hyperparameter Tuning using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(i) for i in np.linspace(start = 100, stop = 1200, num = 12)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']

# Maximum number of levels in tree
max_depth = [int(i) for i in np.linspace(5, 30, num = 6)]
# max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

# Create the random grid (must be in key:value pairs)
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion': ['entropy', 'gini']}
print(random_grid)


In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)
rf_random.fit(x_train, y_train)


In [ ]:
#To print the best params

rf_random.best_params_

In [ ]:
best_random_grid=rf_random.best_estimator_

#Confusion Matrix

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
y_pred=best_random_grid.predict(x_test)
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))
